In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

train_df['label_num'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label_num'] = test_df['label'].map({'ham': 0, 'spam': 1})

train_text = train_df['message'].values
train_labels = train_df['label_num'].values
test_text = test_df['message'].values
test_labels = test_df['label_num'].values

In [ ]:
# Define a function to manually pad or trim text using a special padding symbol
MAX_FEATURES=10000
SEQUENCE_LENGTH=100
def pad_text(text, max_len=SEQUENCE_LENGTH, pad_symbol='<PAD>'):
    words = text.split()
    if len(words) > max_len:
        return ' '.join(words[:max_len])
    else:
        return ' '.join(words + [pad_symbol] * (max_len - len(words)))

train_df['padded_message'] = train_df['message'].apply(lambda x: pad_text(x))
test_df['padded_message'] = test_df['message'].apply(lambda x: pad_text(x))

vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=MAX_FEATURES,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH,
    standardize='lower_and_strip_punctuation'
)

vectorize_layer.adapt(train_df['padded_message'])

train_ds = tf.data.Dataset.from_tensor_slices((train_df['padded_message'].values, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['padded_message'].values, test_labels))

def vectorize_text(text, label):
    return vectorize_layer(text), label

train_ds = train_ds.map(vectorize_text).shuffle(10000).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(vectorize_text).batch(32).prefetch(tf.data.AUTOTUNE)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=MAX_FEATURES + 1, output_dim=64, input_length=SEQUENCE_LENGTH),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(train_ds, validation_data=test_ds, epochs=8)


In [ ]:
# Function to predict messages based on the trained model
# (should return list containing prediction and label, e.g., [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    padded_text = pad_text(pred_text)

    pred_text_vectorized = vectorize_layer([padded_text])

    prediction_score = model.predict(pred_text_vectorized)[0][0]

    label = 'spam' if prediction_score > 0.5 else 'ham'

    return [float(prediction_score), label]

pred_text = "our new mobile video service is live. just install on your phone to start watching."
test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

prediction = predict_message(pred_text)
print(prediction)

for msg in test_messages:
    prediction = predict_message(msg)
    print(f"Message: '{msg}' -> Prediction: {prediction}")


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
